In [ ]:
#1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#en la descripcion sale que los datos nulos estan representados con el 
#caracter '?' por lo que se especifica en read_cv
df = pd.read_csv('https://drive.google.com/uc?id=1YTnA4ikKK6nzd96isNEEVPoP1r_wVdQm&export=download&authuser=0',na_values='?')


In [ ]:
#2
%matplotlib inline

from collections import Counter
sns.set(style='white', context='notebook', palette='deep')

In [ ]:
pip install catboost
#instalamos el modelo catboost


In [ ]:
#instalamos el modelo xgboost
pip install xgboost

In [ ]:
#3
from sklearn.ensemble import AdaBoostClassifier   # Ensamble AdaBoost para regresion
from sklearn.ensemble import GradientBoostingClassifier   # Ensamble Gradient Boosting para regresion
from sklearn.ensemble import RandomForestClassifier   # Ensamble RandomForest para regresion
from sklearn.ensemble import ExtraTreesClassifier     # Ensamble ExtraTrees para regresion

from sklearn.linear_model import LogisticRegression # para usar modelos de regresion logistica  
from sklearn.tree import DecisionTreeClassifier     # para usar modelos de tipo arboles de decision
from sklearn.neighbors import KNeighborsClassifier  # para modelos  Knn 
from sklearn.svm import SVC   # para modelos SVM
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm as lgbm

from sklearn.model_selection import train_test_split  # para partir los datos en conjuntos de entrenamiento y validacion
from sklearn.model_selection import KFold  # para partir la data en k-folds
from sklearn.model_selection import cross_val_score   # para evaluar algoritmos en cross validacion 

from sklearn.metrics import accuracy_score, balanced_accuracy_score   # para manejar metricas de desempeño 
from sklearn.metrics import classification_report  # para hacer reportes de resultados de clasificacion
from sklearn.metrics import confusion_matrix  # para manejar matrices de confusion
from sklearn.metrics import f1_score

In [ ]:
df.head()
#exploramos los que tipos de datos

In [ ]:
df.shape
#el tamaño original del dataset descargado

In [ ]:
df.isnull().sum()
#Buscamos datos nulos en las columnas

In [ ]:
df=df.drop_duplicates()
#eliminamos las filas duplicadas

In [ ]:
df.shape
#el nuevo tamaño del dataset

In [ ]:
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)
#como obervasmos los datos nulos pertenecen a caracteriticos con predominanates
#reemplazamos los valores por la moda de cada uno

In [ ]:
df.isnull().sum()
#verificamos que no existan valores nulos

In [ ]:
df.describe
#descripcion caracteristica de las variables numericas
#aca colocar todas las graficas
#y tambien el outlier

In [ ]:
#A continuacion se mostraran graficas para e

##Columnas categoricas y numericas

In [ ]:
col_numericas = df.select_dtypes('number').columns
col_categoricas = df.select_dtypes(['category','object']).columns
#separamos las variables nuemricas de la categoricas

In [ ]:
col_categoricas

In [ ]:
col_numericas

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [ ]:
for cate in col_categoricas:
  label_encoder.fit(df[cate])
  df[cate] = label_encoder.transform(df[cate])
#usamos label encoder para convertir las valeriables categoricas a numericas

In [ ]:
col_numericas = df.select_dtypes('number').columns
col_categoricas = df.select_dtypes(['category','object']).columns
#verificamos que no hayan variables categoricas

In [ ]:
col_categoricas
#se verifica

In [ ]:
col_numericas

In [ ]:
df.drop(columns=['fnlwgt'], inplace=True)
#como no se tiene informacion de esta columna se procede a eliminar
#ademas de que en la literatura tambien la eliminan

In [ ]:
# df.drop(columns=['sex'], inplace=True)
# df.drop(columns=['race'], inplace=True
# df.drop(columns=['native.country'], inplace=True)
# se penso eliminar estas columnas, sin embargo el score 
# disminuia cuando se eliminaban , por ello se dejar9on

In [ ]:
df.shape
# el tamaño de dataset con una columna eliminda

##Division de la data en train y test

In [ ]:
X_all = df.drop('income',axis=1)
#dropeamos la columna con la variable a predecir (target)
y_all = df['income']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.20, random_state=123)

##Comparacion de los principales modelos de clasificacion


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

pipelines = []
pipelines.append(('LR', make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear'))))
pipelines.append(('DecisionTreeClassifier', make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=6))))
pipelines.append(('KNN', make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5))))
pipelines.append(('SVM', make_pipeline(StandardScaler(), SVC())))
pipelines.append(('AdaBoost', make_pipeline(StandardScaler(), AdaBoostClassifier())))
pipelines.append(('GradientBoosting', make_pipeline(StandardScaler(), GradientBoostingClassifier())))
pipelines.append(('RandomForest', make_pipeline(StandardScaler(), RandomForestClassifier())))
pipelines.append(('ExtraTrees', make_pipeline(StandardScaler(), ExtraTreesClassifier())))
pipelines.append(('XGBClassifier', make_pipeline(StandardScaler(), XGBClassifier())))
pipelines.append(('lgbm.LGBMClassifier', make_pipeline(StandardScaler(), lgbm.LGBMClassifier(silent=False))))
pipelines.append(('CatBoostClassifier', make_pipeline(StandardScaler(), CatBoostClassifier(silent=True))))
# Se usaron los modelos y ensambles de clasificacion vistos en el surso,
# asi como ensamble no vistos como ExtraTrees,XGBClassifier,lgbm.LGBMClassifier,CatBoostClassifier, ya que 
# se usaban frecuentemente en modelos de clasificacion binaria 

In [ ]:
results = []
names = []

#particion en 10-folds para el cross-validation
seed = 7   
kfold = KFold(n_splits=10, random_state=seed, shuffle= True)  

#  evalua cada pipeline en cross-validation
for pipelinename, pipeline in pipelines:
    cv_results = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(pipelinename)
    print("{}: {} ({})".format(pipelinename, cv_results.mean(), cv_results.std()))

In [ ]:
fig = plt.figure()
fig.suptitle('Accuracy  obtenidas en 10-fold-CV')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names,rotation=90)
plt.show()
# Se grafican los accuracys obtenidos en un box-plot para identificar los mejores

In [ ]:

#se hace un nuevo pipeline de los mejores ensables para poder observar
#mejor su desempeño
pipelines = []
pipelines.append(('AdaBoost', make_pipeline(StandardScaler(), AdaBoostClassifier())))
pipelines.append(('GradientBoosting', make_pipeline(StandardScaler(), GradientBoostingClassifier())))
pipelines.append(('XGBClassifier', make_pipeline(StandardScaler(), XGBClassifier())))
pipelines.append(('lgbm.LGBMClassifier', make_pipeline(StandardScaler(), lgbm.LGBMClassifier(silent=False))))
pipelines.append(('CatBoostClassifier', make_pipeline(StandardScaler(), CatBoostClassifier(silent=True))))

In [ ]:
# Se usa de nuevo cross-validation
results = []
names = []


seed = 7   
kfold = KFold(n_splits=10, random_state=seed, shuffle= True)  # especifica el particionador de datos a 10-folds CV

#  evalua cada pipeline en crosvalidacion
for pipelinename, pipeline in pipelines:
    cv_results = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(pipelinename)
    print("{}: {} ({})".format(pipelinename, cv_results.mean(), cv_results.std()))